In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [24]:
df = pd.DataFrame({'A': [1, 2, 3, None, 5], 'B': [1, None, 6, 8, 10], 'C':[2, 6, None, 12, 15], 'D':[5, 10, 15, 20, 25]})
df

,A,B,C,D
0,1.0,1.0,2.0,5
1,2.0,NaN,6.0,10
2,3.0,6.0,NaN,15
3,NaN,8.0,12.0,20
4,5.0,10.0,15.0,25


Выполним построение модели для заполнения пропусков в каждой фиче

In [25]:
def fill_gaps_for_feature(df: pd.DataFrame, target_name: str, gap_feature: str):
    feature_names = df.columns.tolist()
    feature_names.remove(target_name)
    df1 = df[feature_names].copy(deep=True)
    feature_names.remove(gap_feature)
    X_train = df1[df1[gap_feature].notna()][feature_names]
    y_train = df1[df1[gap_feature].notna()][gap_feature]
    X_test = df1[df1[gap_feature].isna()][feature_names]
    X_train = X_train.fillna(X_train.mean())
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    df.loc[df[gap_feature].isna(), gap_feature] = y_pred
    return y_pred.shape[0]

In [26]:
def fill_gaps(df: pd.DataFrame, target_name: str):
    [fill_gaps_for_feature(df=df, target_name=target_name, gap_feature=i) for i in df.columns if df[i].isnull().any()]

In [27]:
fill_gaps(df=df, target_name='D')

Посмотрим на заполненный датасет

In [28]:
df

,A,B,C,D
0,1.000000,1.000000,2.000000,5
1,2.000000,3.570868,6.000000,10
2,3.000000,6.000000,9.682344,15
3,4.137263,8.000000,12.000000,20
4,5.000000,10.000000,15.000000,25


Теперь используем его для построения модели для целевого признака

In [34]:
feature_names = ['A', 'B', 'C']
target_name = 'D'
X_train = df.loc[[0, 1, 2], feature_names]
y_train = df.loc[[0, 1, 2], target_name]
X_test = df.loc[3:, feature_names]
y_test = df.loc[3:, target_name]
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_pred)

[20.84180839 25.11921345]
